In [1]:
import os
import datetime
import logging
import shutil
import pandas as pd
import glob
import csv
#from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import time
import threading
import os
import shutil
import time
import multiprocessing
import re

from FUNCTION import make_top_protein, fill_water_ions, energy_min, make_new_minim_nvt_npt, make_new_minim_config_samd, run_md
from FUNCTION import files_gmxmmpbsa, gmx_mmpbsa, Data_Analysis_Pre, Data_Analysis_Cal, clean_for_each_cycle

#PROJECT_ROOT = os.path.dirname(os.path.abspath(__file__))
PROJECT_ROOT = os.getcwd()

DATA_DIR = os.path.join(PROJECT_ROOT, "DATA")
VMD_DIR = os.path.join(PROJECT_ROOT, "VMD_FUNCTION")
FUNCTION_DIR = os.path.join(PROJECT_ROOT, "FUNCTION")
FORCE_FIELD_PATH = os.path.join(PROJECT_ROOT, "FORCE_FIELD")
MMPBSA_INFILE_PATH = os.path.join(PROJECT_ROOT, "gmx_mmpbsa_in")
# pdb file
protein_infile = "HLA_BiAB_protein_50ns" 
#protein_infile = "mtbind"
protein_file_path = os.path.join(DATA_DIR, f"{protein_infile}.pdb")

# MDP files
ions_mdp_file = "ions"
minim_mdp_file = "minim"
nvt_mdp_file = "NVT"
npt_mdp_file = "NPT"
samd_mdp_file = "SAMD"
md_mdp_file = "EngComp_ff14sb_custom"
only_protein_md_mdp_file = "Protein_EngComp_ff14sb_custom"

ions_mdp_path = os.path.join(DATA_DIR, f"{ions_mdp_file}.mdp")
minim_mdp_path = os.path.join(DATA_DIR, f"{minim_mdp_file}.mdp")
nvt_mdp_path = os.path.join(DATA_DIR, f"{nvt_mdp_file}.mdp")
npt_mdp_path = os.path.join(DATA_DIR, f"{npt_mdp_file}.mdp")
samd_mdp_path = os.path.join(DATA_DIR, f"{samd_mdp_file}.mdp")
md_mdp_path = os.path.join(DATA_DIR, f"{md_mdp_file}.mdp")
only_protein_md_mdp_path = os.path.join(DATA_DIR, f"{only_protein_md_mdp_file}.mdp")



def create_output_directory():
    
    current_dir = os.getcwd()
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir_name = f"output_{timestamp}"
    output_dir_path = os.path.join(current_dir, output_dir_name)
    
    os.mkdir(output_dir_path)
    print(f"Created directory: {output_dir_path}")

    os.chdir(output_dir_path)

    
    
    return output_dir_path

ROOT_OUTPUT = create_output_directory()

logging.basicConfig(
    filename = "OUTPUT.out",
    level = logging.INFO,
    format="%(asctime)s - %(levelname)s -%(message)s"
)

logging.info(f"PATH: {ROOT_OUTPUT}")

# create configuration folder
configuration_path = os.path.join(os.getcwd(),"configuration")
os.mkdir(configuration_path)
print(f"Create directory: {configuration_path}")
os.chdir(configuration_path)
logging.info("####Begin with configuration####")

# build folders
# cycle_num, MAKE IT 1 NOW, NEED TO CHANGE
cycle_num = 2
def build_folders(current_dir, cycle_num):
    # Create folder for each cycle
    folders  ={}
    
    for cycle_n in range (1,cycle_num + 1):
        folder_name = f"cycle{cycle_n}_MD"
        folder_path = os.path.join(current_dir, folder_name)
        os.makedirs(folder_path, exist_ok = True)
        folders[f"cycle{cycle_n}_MD"] = folder_path

    folders["repository"] = os.path.join(current_dir,"REPOSITORY")
    folders["TEMP_FILES_FOLDER"] = os.path.join(current_dir,"TEMP_FILES_FOLDER")
    folders["REMOVED_FILES_FOLDER"] = os.path.join(current_dir,"REMOVED_FILES_FOLDER")
    folders["results"] = os.path.join(current_dir,"RESULTS")

    for folder in folders.values():
        os.makedirs(folder,exist_ok = True)
  
    header = [
    "#RUNnumber", "DeltaG(kJ/mol)", "Coul(kJ/mol)", "vdW(kJ/mol)",
    "PolSol(kJ/mol)", "NpoSol(kJ/mol)", "ScoreFunct", "ScoreFunct2",
    "Canonica_AVG", "MedianDG", "DeltaG_2s", "dG_PotEn"]

    df = pd.DataFrame(columns=header)
    results_file_path = os.path.join(folders["results"], "MoleculesResults.dat")
    df.to_csv(results_file_path, sep='\t', index=False, header=True)

    return folders
    

current_dir = os.getcwd()
folders = build_folders(current_dir,cycle_num)

# generating a topology and build box
make_top_protein(protein_file_path, "amber99sb-ildn", "tip3p", "system", "topol")

# cp system.pdb {protein_infile}.pdb in current folder
source = os.path.join(current_dir, "system.pdb")
destination = os.path.join(current_dir, f"{protein_infile }.pdb")
try:
    shutil.copy(source,destination)
except Exception:
    print("Copy system.pdb failed.")

## OC2 DOESN'T NORMAL ONE, NEED TO CHANGE
def add_ter_to_pdb(pdb_file_name):
    
    temp_file_name = f"{pdb_file_name}_temp"  
    with open(pdb_file_name, 'r') as f:
        lines = f.readlines()

    new_lines = []  
    for i, line in enumerate(lines):
        new_lines.append(line)
        if "OC2" in line:
            if i + 1 >= len(lines) or not lines[i + 1].startswith("TER"):
                new_lines.append("TER\n")

    with open(temp_file_name, 'w') as f:
        f.writelines(new_lines)

    os.rename(temp_file_name, pdb_file_name)

add_ter_to_pdb(f"{protein_infile }.pdb")

def replace_his_residues_flexible(input_pdb, output_pdb):
    with open(input_pdb, "r", encoding="utf-8") as infile, open(output_pdb, "w", encoding="utf-8") as outfile:
        for line in infile:
            if line.startswith("ATOM"):
                match = re.match(r"(.{17})(HISE|HISD|HISP)(.*)", line)
                if match:
                    line = f"{match.group(1)}{'HIS':<4}{match.group(3)}"
            
            outfile.write(line.rstrip('\n') + '\n')

output_pdb = os.path.join(ROOT_OUTPUT, f"{protein_infile}.pdb")
replace_his_residues_flexible(f"{protein_infile}.pdb",output_pdb)

# Adding water and ions
fill_water_ions("system", "topol", ions_mdp_path)
# Energy Minimiization
energy_min(minim_mdp_path, "system_ions", "topol", "system_compl")

# Nvt and Npt
sequence = 0
make_new_minim_nvt_npt("system_compl_minim.gro", nvt_mdp_path, npt_mdp_path, "system_equil", 0)

# Move .cpt, .top, and .itp files to repository folder
for file_pattern in [f"{current_dir}/*.cpt", f"{current_dir}/*.top", f"{current_dir}/*.itp"]:
    for file in glob.glob(file_pattern):
        shutil.move(file, folders["repository"])

# Move specific files to repository folder
shutil.move(f"{current_dir}/{protein_infile}.pdb", folders["repository"])
shutil.move(f"{current_dir}/system_compl_minim.gro", folders["repository"])
shutil.move(f"{current_dir}/system_equil.gro", folders["repository"])


# Move temp* and *out files to removed files folder
for file in glob.glob("./*temp*.*") + glob.glob("./*.temp") + glob.glob("./*out"):
    shutil.move(file, folders["REMOVED_FILES_FOLDER"])

# Remove files with # in their name
for file in glob.glob("./#*"):
    os.remove(file)

Created directory: /home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458
Create directory: /home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration

Statistics over 50001 steps [ 0.0000 through 50.0000 ps ], 1 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 293.517        6.4    19.5716    35.7128  (K)

Statistics over 50001 steps [ 0.0000 through 100.0000 ps ], 2 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                   -2.00041         --     90.326    18.5344  (bar)
Density                     1011.06         --    1.38924  0.0012282  (kg/m^3)


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration
Command line:
  gmx energy -f NVT.edr -o temp_NVT.xvg

Opened NVT.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Position-Rest.  12  Potential     
 13  Kinetic-En.     14  Total-Energy    15  Conserved-En.   16  Temperature   
 17  Pres.-DC        18  Pressure        19  Constr.-rmsd    20  Vir-XX        
 21  Vir-XY          22  Vi

In [2]:

def MD_for_each_cycle(work_dir, cycle_number,input_structure_file,samd_mdp_path,output_gro, sequence, md_mdp_path, tpr_file, trj_name):
    print("start MD in MD function")
    #cycle_number = 1
    #while cycle_number <= cycle_num:
        
    cycle_MD_path = folders[f"cycle{cycle_number}_MD"] 
    os.chdir(work_dir)
    shutil.copy(os.path.join(folders["repository"], "system_equil.gro"), "./")
    shutil.copy(os.path.join(folders["repository"], "topol.top"), "./")

    for itp_file in glob.glob(os.path.join(folders["repository"], "*rotein_chain_*.itp")):
        shutil.copy(itp_file, "./")

    for itp_file in glob.glob(os.path.join(folders["repository"], "posres_*.itp")):
        shutil.copy(itp_file, "./")

    for cpt_file in glob.glob(os.path.join(folders["repository"], "*NPT*.cpt")):
        shutil.copy(cpt_file, "./")

    #make_new_minim_config_samd("system_equil.gro", samd_mdp_path, "system_Compl_MDstart", 0)
    make_new_minim_config_samd(input_structure_file, samd_mdp_path, output_gro, sequence)
    #run_md(md_mdp_path,"system_Compl_MD", "traj_MD", 0, 1)
    run_md(md_mdp_path, tpr_file, trj_name, sequence, cycle_number)
    shutil.copy("system_Compl_MD.gro", f"LastFrame_cycle{cycle_number}.gro")
    #cycle_number += 1


def gmx_mmpbsa_for_each_cycle(work_dir, cycle_number,only_protein_md_mdp_path,VMD_DIR,temp_files_folder, FORCE_FIELD_PATH, MMPBSA_INFILE_PATH, REMOVED_FILES_FOLDER, results_folder, repository_folder, current_conf_path):
    #cycle_number = 1
    #while cycle_number <= cycle_num:
    ConfName = f"cycle{cycle_number}"
    RootName = f"cycle{cycle_number}_BE"
    cycle_number_MD_FOLDER = folders[f"cycle{cycle_number}_MD"]
    # 输出相关信息
    print(f"Cycle Number: {cycle_number}")
    print(f"Configuration Name: {ConfName}")
    print(f"Root Name: {RootName}")
    print(f"MD Folder Path: {cycle_number_MD_FOLDER}")
    #os.chdir(cycle_number_MD_FOLDER )
    os.chdir(work_dir)
        
    repository_pdb_file = os.path.join(repository_folder, f"{protein_infile}.pdb")
    #startingFrameGMXPBSA="2000"
    # make files for gmx_mmpbsa
    # files_gmxmmpbsa(starting_gro_file, repository_pdb_file, trj_file, tpr_file, top_file, mdp_name, root_name, conf_name, vmd_function_folder, temp_files_folder)

    files_gmxmmpbsa("system_Compl_MD", repository_pdb_file, "traj_MD", "system_Compl_MD", "topol", only_protein_md_mdp_path, RootName, ConfName, VMD_DIR, temp_files_folder, cycle_number)
    # get number of frames
    try:
        with open("trj_check.out", "r") as file:
            number_of_frames = next(
                (line.split()[1] for line in file if line.startswith("Step")), None
            )
    except FileNotFoundError:
        print(f"Error: File trj_check.out not found.")
        number_of_frames = None
    #conda_activate_path="/home/bio/ls/bin"
    conda_path = shutil.which("conda")
    conda_activate_path = os.path.dirname(conda_path)
    conda_gmxmmpbsa_name="gmxMMPBSA"
    forcefield="amber99sb-ildn"
    #FORCE_FIELD_PATH = "/home/bio/Desktop/jupyter_test/antibody_test/FORCE_FIELD"
    mmpbsa_inFILE="mmpbsa_LinearPB_amber99SB_ILDN.in"
    #MMPBSA_INFILE_PATH = "/home/bio/Desktop/jupyter_test/antibody_test/gmx_mmpbsa_in"
    np_value = 32
    #gmx_mmpbsa(1, conda_activate_path, conda_gmxmmpbsa_name, cycle_number_MD_FOLDER, ConfName, RootName, forcefield, FORCE_FIELD_PATH, 
    #             mmpbsa_inFILE, MMPBSA_INFILE_PATH , np_value, number_of_frames)
    gmx_mmpbsa(cycle_number, conda_activate_path, conda_gmxmmpbsa_name, cycle_number_MD_FOLDER, ConfName, RootName, forcefield, FORCE_FIELD_PATH, mmpbsa_inFILE, MMPBSA_INFILE_PATH, np_value, number_of_frames)
    # data analysis
    NUMframe = "all"
    Data_Analysis_Pre(cycle_number_MD_FOLDER, REMOVED_FILES_FOLDER, NUMframe)
    Data_Analysis_Cal(cycle_number, results_folder)
    # clean and move files
    clean_for_each_cycle(cycle_number, repository_folder, cycle_number_MD_FOLDER, RootName, REMOVED_FILES_FOLDER, current_conf_path)
        # 更新周期号及相关变量
    #cycle_number += 1
    #conf_name = f"cycle{cycle_number}"
    #root_name = f"cycle{cycle_number}_BE"
    #cycle_number_md_folder = os.path.join(current_conf_path, f"cycle{cycle_number}_MD")
    
 


cycle_num =2
sequence = 0
md_args = ("system_equil.gro",samd_mdp_path,"system_Compl_MDstart", sequence, md_mdp_path, "system_Compl_MD", "traj_MD")
gmx_args = (only_protein_md_mdp_path,VMD_DIR,folders["TEMP_FILES_FOLDER"], FORCE_FIELD_PATH, MMPBSA_INFILE_PATH, folders["REMOVED_FILES_FOLDER"], folders["results"], folders["repository"], configuration_path)




def run_cycle(cycle_number, cycle_num, md_args, gmx_args):
    """
    处理当前 cycle 的 gmx_mmpbsa 和下一个 cycle 的 MD。
    """
    process = []

    # 启动当前 cycle 的 gmx_mmpbsa
    gmx_process = multiprocessing.Process(target=gmx_mmpbsa_for_each_cycle, args=(folders[f"cycle{cycle_number}_MD"],cycle_number, *gmx_args))
    process.append(gmx_process)
    gmx_process.start()

    # 如果还有下一个 cycle，启动下一个 cycle 的 MD
    if cycle_number < cycle_num:
        next_md_process = multiprocessing.Process(target=MD_for_each_cycle, args=(folders[f"cycle{cycle_number+1}_MD"], cycle_number + 1, *md_args))
        process.append(next_md_process)
        next_md_process.start()

    # 等待所有线程完成
    for p in process:
        p.join()



# 首先启动第一个 cycle 的 MD
MD_for_each_cycle(folders["cycle1_MD"],1, *md_args)

# 然后依次处理每个 cycle 的 gmx_mmpbsa 和下一个 MD
for cycle_number in range(1, cycle_num + 1):
    run_cycle(cycle_number, cycle_num, md_args, gmx_args)


start MD in MD function


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration/cycle1_MD
Command line:
  gmx energy -f SAMD.edr -o press_SAMD.xvg

Opened SAMD.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  Box-Z   


Statistics over 175001 steps [ 0.0000 through 350.0000 ps ], 3 data sets
All statistics are over 1751 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 340.007         13    27.1496    -51.099  (K)
Pressure                   0.938763         11     97.331   -43.6104  (bar)
Density                     981.647         13      28.14    48.9855  (kg/m^3)
Selected 1: 'Protein'
Selected 1: 'Protein'
SAMD completed successfully!
17:56:44 -- Running MD 


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration/cycle1_MD
Command line:
  gmx energy -f PROD.edr -o PROD0.xvg

Opened PROD.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  Box-Z        


Statistics over 2500001 steps [ 0.0000 through 5000.0000 ps ], 3 data sets
All statistics are over 25001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 309.957      0.015    1.23366 -0.0101229  (K)
Pressure                    1.39043        1.1    99.8682    2.62941  (bar)
Density                     1012.11      0.076    2.03365   0.321974  (kg/m^3)


...

/home/bio/Desktop/jupyter_test/antibody_test/FUNCTION/Data_Analysis.py:173: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(input_file, delim_whitespace=True, header=None)
/home/bio/Desktop/jupyter_test/antibody_test/FUNCTION/Data_Analysis.py:196: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  DeltaG_temp = row[1]  # 使用列名访问 DeltaG


Cycle Number: 2
Configuration Name: cycle2
Root Name: cycle2_BE
MD Folder Path: /home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration/cycle2_MD
		--running MAKE_NDX to create index.ndx..


                     :-) GROMACS - gmx make_ndx, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration/cycle2_MD
Command line:
  gmx make_ndx -f system_Compl_MD.gro -o index.ndx


Reading structure file

GROMACS reminds you: "Product of optimism and knowledge is a constant." (Lev Landau)



Going to read 0 old index file(s)
Analysing residue names:
There are:   413    Protein residues
There are: 23061      Water residues
There are:   141        Ion residues
Analysing Protein...

  0 System              : 75722 atoms
  1 Protein             :  6398 atoms
  2 Protein-H           :  3292 atoms
  3 C-alpha             :   413 atoms
  4 Backbone            :  1239 atoms
  5 MainChain           :  1656 atoms
  6 MainChain+Cb        :  2035 atoms
  7 MainChain+H         :  2063 atoms
  8 SideChain           :  4335 atoms
  9 SideChain-H         :  1636 atoms
 10 Prot-Masses         :  6398 atoms
 11 non-Protein         : 69324 atoms
 12 Water               : 69183 atoms
 13 SOL                 : 69183 atoms
 14 non-Water           :  6539 atoms
 15 Ion                 :   141 atoms
 16 Water_and_ions      : 69324 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't

/home/bio/Desktop/jupyter_test/antibody_test/FUNCTION/Data_Analysis.py:173: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(input_file, delim_whitespace=True, header=None)
/home/bio/Desktop/jupyter_test/antibody_test/FUNCTION/Data_Analysis.py:196: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  DeltaG_temp = row[1]  # 使用列名访问 DeltaG


In [3]:
last_cycle_MD_FOLDER = os.path.join(folders["repository"],f"cycle{cycle_number}_MD")

In [4]:
last_cycle_gro = os.path.join(last_cycle_MD_FOLDER,f"LastFrame_cycle{cycle_number}.gro")

In [5]:
shutil.copy(last_cycle_gro, os.path.join(folders["repository"],f"LastFrame_cycle{cycle_number}.gro"))

'/home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration/REPOSITORY/LastFrame_cycle2.gro'

In [6]:
logging.info(f"Making the starting PDB for the next Mutation from LastFrame_cycle{cycle_number}.gro")

In [7]:
def GRO_to_PDB(pathGRO, fileNameGRO, pathPDB, fileNamePDB, FileNamePDB_OUT, vmd_function_folder, temp_files_folder):
    """
    Converts a GRO file to a PDB file using VMD.

    Parameters:
        pathGRO (str): Path to the input GRO file.
        fileNameGRO (str): Name of the GRO file.
        pathPDB (str): Path to the output PDB file. 
        fileNamePDB (str): Name of the output PDB file.
        temp_files_folder (str): Temporary files folder.
        vmd_executable (str): Path to VMD executable.
    """
    
    vmd = shutil.which("vmd")
    if not vmd:
        raise FileNotFoundError("VMD not found. please ensure VMD is installed and added to PATH")
    # Paths and filenames
    vmd_script = os.path.join(temp_files_folder, "vmd_GRO_to_PDB.tcl")
    log_file = os.path.join(temp_files_folder, "vmd_GRO_to_PDB.out")
    function_tcl = os.path.join(vmd_function_folder, "VMD_function_GRO_to_PDB.tcl")

    # Ensure paths and required files exist
    if not os.path.exists(temp_files_folder):
        os.makedirs(temp_files_folder)
    if not os.path.isfile(function_tcl):
        raise FileNotFoundError(f"Function script missing: {function_tcl}")

    if not FileNamePDB_OUT:
        FileNamePDB_OUT = fileNameGRO

    # Write VMD TCL script
    tcl_content = f"""
    variable _pathGRO "{pathGRO}"
    variable _FileNameGRO "{fileNameGRO}"
    variable _pathPDB "{pathPDB}"
    variable _FileNamePDB "{fileNamePDB}"
    variable _FileNamePDB_OUT "{FileNamePDB_OUT}"
    """
    with open(vmd_script, 'w') as f:
        f.write(tcl_content)
        f.write(open(function_tcl).read())

    # Execute VMD
    result = subprocess.run(
        [vmd, "-dispdev", "none", "-e", vmd_script],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )

    # Log results
    with open(log_file, 'w') as log:
        log.write(result.stdout + result.stderr)

    if result.returncode != 0:
        raise RuntimeError(f"VMD execution failed. Check log: {log_file}")

    print(f"GRO to PDB completed successfully: {fileNamePDB}")

In [8]:
# os.path.join(folders["repository"],f"LastFrame_cycle{cycle_number}.gro")
import os
import subprocess
import sys
import shutil
import glob
import logging

os.chdir(configuration_path)
repository_pdb_file = os.path.join(folders["repository"], f"{protein_infile}.pdb")

pathGRO = folders["repository"]
fileNameGRO = f"LastFrame_cycle{cycle_number}"
pathPDB = os.path.dirname(repository_pdb_file)
pdb_name_with_extension = os.path.basename(repository_pdb_file) #xxxx.pdb
pdb_name_without_extension = os.path.splitext(pdb_name_with_extension)[0] #xxxx
fileNamePDB = pdb_name_without_extension
FileNamePDB_OUT = f"LastFrame_cycle{cycle_number}"
GRO_to_PDB(pathGRO, fileNameGRO, pathPDB, fileNamePDB, FileNamePDB_OUT, VMD_DIR, folders["TEMP_FILES_FOLDER"])

GRO to PDB completed successfully: HLA_BiAB_protein_50ns


In [9]:
ROOT_OUTPUT

'/home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458'

In [10]:
last_cycle_pdb = os.path.join(folders["repository"], f"LastFrame_cycle{cycle_number}.pdb")
add_ter_to_pdb(last_cycle_pdb)

In [34]:


def replace_his_residues_flexible(input_pdb, output_pdb):
    with open(input_pdb, "r", encoding="utf-8") as infile, open(output_pdb, "w", encoding="utf-8") as outfile:
        for line in infile:
            # 判断是否是 ATOM 或 HETATM 行
            if line.startswith("ATOM") or line.startswith("HETATM"):
                # 使用正则表达式提取残基类型
                match = re.match(r"(.{17})(HISE|HISD|HISP)(.*)", line)
                if match:
                    # 替换成 HIS，确保格式对齐
                    line = f"{match.group(1)}{'HIS':<4}{match.group(3)}"
            
            # 写入每行时确保换行符
            outfile.write(line.rstrip('\n') + '\n')

In [11]:
output_last_cycle_pdb = os.path.join(ROOT_OUTPUT, f"Mutant{sequence}_cycle{cycle_number}_LastFrameMD.pdb")
replace_his_residues_flexible(last_cycle_pdb,output_last_cycle_pdb)

In [12]:
# update protein_infile
protein_infile = f"Mutant{sequence}_cycle{cycle_number}_LastFrameMD"
logging.info("Making the average of the cycles results.")

In [43]:
def Data_Analysis_Cal_child(input_file, output_file, Data_Analysis_Signal = True):
    # 假设文件数据为 tab-separated
    
    df = pd.read_csv(input_file, delim_whitespace=True, header=None)
    # 为了清晰，假设文件列对应以下字段
    if Data_Analysis_Signal == True:
        df.columns = ['frame', 'DeltaG', 'Coul', 'VdW', 'PolSol', 'NpoSol']
    else:
        df.columns = ['frame', 'DeltaG', 'Coul', 'VdW', 'PolSol', 'NpoSol', 'SF1', 'SF2', 'Canonical_AVG', 'MedianDG', 'DeltaG_2s']

    # 初始化变量
    Population = len(df)
    DeltaG = df['DeltaG'].sum()
    Coul = df['Coul'].sum()
    VdW = df['VdW'].sum()
    PolSol = df['PolSol'].sum()
    NpoSol = df['NpoSol'].sum()

    # 计算 SF1 和 SF2
    df['SF1'] = (df['Coul'] / 10) - (df['PolSol'] / 10) + (df['NpoSol'] * 10)
    df['SF2'] = (3 * df['Coul']) + df['PolSol']

    # 初始化变量
    Canonical_AVG = 0.0
    Canonical_AVG_w = 0.0

    for _, row in df.iterrows():
        # 尝试将 DeltaG 转换为浮动类型
        DeltaG_temp = row[1]  # 使用列名访问 DeltaG
        # 计算指数加权值
        weight = math.exp(-int(DeltaG_temp / 2.479))
        # 累加计算
        Canonical_AVG += DeltaG_temp * weight
        Canonical_AVG_w += weight

    # 归一化
    if Canonical_AVG_w != 0:
        Canonical_AVG /= Canonical_AVG_w

    # 平均值计算
    mean_DeltaG = DeltaG / Population
    mean_Coul = Coul / Population
    mean_VdW = VdW / Population
    mean_PolSol = PolSol / Population
    mean_NpoSol = NpoSol / Population

    mean_SF1 = mean_Coul / 10 - mean_PolSol / 10 + mean_NpoSol * 10
    mean_SF2 = (3 * mean_Coul) + mean_PolSol


    # 计算标准差
    std_DeltaG = np.std(df['DeltaG'], ddof=1)
    std_Coul = np.std(df['Coul'], ddof=1)
    std_VdW = np.std(df['VdW'], ddof=1)
    std_PolSol = np.std(df['PolSol'], ddof=1)
    std_NpoSol = np.std(df['NpoSol'], ddof=1)
    std_SF1 = std_Coul / 10 - std_PolSol / 10 + std_NpoSol * 10
    std_SF2 = (3 * std_Coul) + std_PolSol

    # 计算 DeltaG 在2sigma内的值
    DeltaG_2s = df[(np.abs(df['DeltaG'] - mean_DeltaG) < 2 * std_DeltaG)]['DeltaG'].mean()

    # 计算中位数
    median_DeltaG = df['DeltaG'].median()
    # 计算标准差
    std_DeltaG = np.std(df['DeltaG'], ddof=1)  # ddof=1 表示使用样本标准差公式（贝塞尔校正）


    # 创建警告信息列表
    warnings = []

    # 遍历每一行，检查是否超出 2 sigma
    for i, row in df.iterrows():
        var = np.abs(row['DeltaG'] - mean_DeltaG)
        if var >= 2 * std_DeltaG:
            warnings.append(f"# WARNING: frame {row['frame']} is out of 2 sigma!!")

    # 输出处理结果
    with open(output_file, 'w') as f:
        # 写入头部信息
        f.write("# SF1=Coulomb/10-PolarSolvation/10+Non-PolarSolvation*10\n")
        f.write("# SF2=3*Coulomb+PolarSolvation\n")
        f.write("# C_AVG=norm(SUM Gi*e^BGi)\n")
        f.write(f"#frame\tDeltaG(kJ/mol)\tCoul(kJ/mol)\tVdW(kJ/mol)\tPolSol(kJ/mol)\tNpoSol(kJ/mol)\tSF1\tSF2\n")

        # 写入数据
        for _, row in df.iterrows():
            f.write(f"{row['frame']:<10}{row['DeltaG']:>12.3f}{row['Coul']:>13.3f}{row['VdW']:>13.3f}{row['PolSol']:>13.3f}{row['NpoSol']:>13.3f}{row['SF1']:>13.3f}{row['SF2']:>13.3f}\n")
    
         # 写入警告信息
        for warning in warnings:
            f.write(f"{warning}\n")   
   
        # 写入汇总信息表头
        f.write("\n# FINAL RESULTS\n")
        f.write(f"#frame\t{'DeltaG(kJ/mol)':>15}\t{'Coul(kJ/mol)':>15}\t{'VdW(kJ/mol)':>15}\t{'PolSol(kJ/mol)':>15}\t{'NpoSol(kJ/mol)':>15}\t{'SF1':>15}\t{'SF2':>15}\t{'Canonical_AVG':>15}\t{'MedianDeltaG(kJ/mol)':>15}\t{'DeltaG_2s(kJ/mol)':>15}\n")
        # 写入汇总结果
        f.write(f"#AVG\t{mean_DeltaG:>15.1f}\t{mean_Coul:>15.1f}\t{mean_VdW:>15.1f}\t{mean_PolSol:>15.1f}\t{mean_NpoSol:>15.1f}\t{mean_SF1:>15.1f}\t{mean_SF2:>15.1f}\t{Canonical_AVG:>15.1f}\t{median_DeltaG:>15.1f}\t{DeltaG_2s:>15.1f}\n")
        f.write(f"#STD\t{std_DeltaG:>15.1f}\t{std_Coul:>15.1f}\t{std_VdW:>15.1f}\t{std_PolSol:>15.1f}\t{std_NpoSol:>15.1f}\t{std_SF1:>15.1f}\t{std_SF2:>15.1f}\t{"nan":>15}\t{"nan":>15}\t{std_DeltaG:>15.1f}\n")    


In [44]:
import csv
import os
import shutil
import logging
import numpy as np
import pandas as pd
import math

In [46]:
print(csv)

<module 'csv' from '/home/bio/ls/lib/python3.12/csv.py'>


In [13]:
all_cycle_data = "All_cycle_data.out"
MoleculesResults_data = os.path.join(folders["results"], "MoleculesResults.dat")
data_analysis_temp = "DataAnalysis_temp.csv"

if os.path.exists(all_cycle_data):
    os.remove(all_cycle_data)

flag_header = True

with open(MoleculesResults_data, 'r') as infile, open(all_cycle_data, 'w') as outfile:
    for line in infile:
        if flag_header == True:
            # 写入标题行，保留格式
            outfile.write(f"#{'configNum':<10} \t{line}")
            flag_header = False
        else:
            # 写入其他数据行
            outfile.write(f"{'avg':<10} \t{line}")


In [42]:
# 使用 pandas 读取文件并提取第二列及以后的数据
df = pd.read_csv(all_cycle_data, sep='\t')  # 读取数据，使用制表符分隔
df_filtered = df.iloc[:, 1:]  # 提取第二列及后续列

# 将结果保存为 CSV 格式
df_filtered.to_csv(data_analysis_temp, sep = '\t', index=False,header = False)


In [48]:
Data_Analysis_Signal = False
import math
import numpy as np
Data_Analysis_Cal_child(data_analysis_temp, "AllData.temp", Data_Analysis_Signal)

/tmp/ipykernel_31639/1820993772.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(input_file, delim_whitespace=True, header=None)
/tmp/ipykernel_31639/1820993772.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  DeltaG_temp = row[1]  # 使用列名访问 DeltaG


In [53]:
'''
# 格式化输出文件
with open("AllData.temp", 'r') as infile, open("AllData.temp", 'w') as outfile:
    for line in infile:
        # 替换格式，添加额外的制表符
        line = line.replace("#AVG", "#AVG\t\t").replace("#STD", "#STD\t\t")
        outfile.write(line)
'''
frame_count = 0
# 提取 #AVG 和 #STD 的行并追加到 AllData.out
with open("AllData.temp", 'r') as temp_file, open(all_cycle_data, 'a') as outfile:
    for line in temp_file:

        if line.startswith("#frame"):
            frame_count +=1
            if frame_count ==2:
                outfile.write(line)
        if line.startswith("#AVG") or line.startswith("#STD"):
            outfile.write(line)
'''
# 移动临时文件到指定目录
for temp_file in [all_data_temp, data_analysis_temp]:
    shutil.move(temp_file, os.path.join(removed_files_folder, os.path.basename(temp_file)))
'''

'\n# 移动临时文件到指定目录\nfor temp_file in [all_data_temp, data_analysis_temp]:\n    shutil.move(temp_file, os.path.join(removed_files_folder, os.path.basename(temp_file)))\n'

In [63]:
frame = []
avg = []
std = []
with open(all_cycle_data, 'r') as infile:
# 逐行解析文件内容
    for line in infile:
        if line.startswith("#frame"):
            frame = line.strip().split()[1:]
        elif line.startswith("#AVG"):
            avg = line.strip().split()[1:]
        elif line.startswith("#STD"):
            std = line.strip().split()[1:]
if frame and avg and std:
    output_lines = ["Results for Configuation"]
    output_lines += [f"{frame[i]}: {avg[i]} +- {std[i]} kJ/mol"
                    for i in range(len(frame))
                   ]
    logging.info("\n".join(output_lines))

In [64]:
shutil.move("AllData.temp", folders["REMOVED_FILES_FOLDER"])

'/home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration/REMOVED_FILES_FOLDER/AllData.temp'

In [65]:
shutil.move(data_analysis_temp, folders["REMOVED_FILES_FOLDER"])

'/home/bio/Desktop/jupyter_test/antibody_test/output_20241205_175458/configuration/REMOVED_FILES_FOLDER/DataAnalysis_temp.csv'